In [195]:
import numpy as np
import pandas as pd
import scipy as sp
import plotly.express as px
import networkx as nx
import degroot as dg
from tqdm import tqdm

In [196]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
# Parameters
dims = 10
erdos_reni_p = 0.5
n_initial_conditions_attempts = 100
max_edge_value = 10
init_belief = np.random.rand(dims)
correct_belief = 0.7
niter_degroot = 1000
niter_search = 100_000
convergence_bar = 10_000

In [219]:
def generate_initial_conditions(n: int, p: float, n_attempts: int):
    initial_conditions = []
    for i in range(n_attempts):
        m = nx.adjacency_matrix(nx.erdos_renyi_graph(n=n, p=p)).todense()
        if np.min(np.sum(m, axis=1)) == 0:
            continue
            
        m = np.random.randint(1, max_edge_value + 1, size=(n, n)) * m
        initial_conditions.append(m)
    
    unique_initial_conditions = np.unique([s.flatten() for s in initial_conditions], axis=0)
    unique_initial_conditions = [s.reshape(dims, dims) for s in unique_initial_conditions]
    print('Number of unique initial conditions:', len(unique_initial_conditions))
    return unique_initial_conditions

In [220]:
init_cond = generate_initial_conditions(n=dims, p=erdos_reni_p, n_attempts=n_initial_conditions_attempts)

Number of unique initial conditions: 98


In [221]:
results = []
for x in tqdm(init_cond):
    res = dg.myopic_search(m0=x, niter_search=niter_search, convergence_bar=convergence_bar, max_edge_value=max_edge_value, initial_belief=init_belief, correct_belief=correct_belief, niter_degroot=niter_degroot)
    results.append(res)

100%|██████████| 98/98 [01:07<00:00,  1.46it/s]


In [222]:
meta_data = pd.DataFrame([{'final_error': r.error, 'initial_error': r.initial_error, 'converged': r.converged, 'iterations': r.iterations} for r in results])
px.scatter(meta_data, x='initial_error', y='final_error', color='converged')

In [210]:
px.histogram(meta_data, x='iterations', color='converged')

In [211]:
flat_solutions = np.array([r.m.flatten() for r in results])
dist = sp.spatial.distance.pdist(flat_solutions, metric='hamming')
hist, bins = np.histogram(dist, bins=dims)
px.scatter(x=bins[1:], y=hist)